# Iris - classification using keras and tensorflow probability
*co-created by Stephen Haddad and ChatGPT*


aims of notebook
* demonstrate use of tensorflow  for probabilistic classification on iris benchmasrk dataset


Other benchmasrk datasets might be tried in this or another notebook
https://drive.google.com/drive/folders/1FeZ_yLilvmE1qUp2QRv_wGSfAsJroMIW?usp=share_link

Explanation from ChatGPT:
> In this example, we use Keras to define the variational distribution that approximates the posterior over the parameters of the Gaussian mixture model. The Keras model consists of two hidden layers with dropout regularization and three output layers for the means, scales, and mixture weights of the Gaussian mixture model. We then use TensorFlow Probability's `fit_surrogate_posterior`


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp


In [2]:

# Load the Iris dataset
iris = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
                   header=None, names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species'])
iris = iris.drop(columns='species')

# Define a probabilistic model using TensorFlow Probability
def model_fn():
    # Priors over the parameters of a Gaussian mixture model
    weights = tfp.layers.DenseFlipout(3, activation='softmax', name='weights')(tf.zeros((1, 4)))
    means = tfp.layers.DenseFlipout(3, name='means')(tf.zeros((1, 4)))
    scales = tfp.layers.DenseFlipout(3, activation='softplus', name='scales')(tf.zeros((1, 4)))

    # Mixture components
    dist = tfp.distributions.MixtureSameFamily(
        mixture_distribution=tfp.distributions.Categorical(probs=weights),
        components_distribution=tfp.distributions.Normal(loc=means, scale=scales))
    return dist

# Define a Keras model for the variational distribution using TensorFlow Probability
def variational_fn():
    # Inputs to the model
    inputs = tf.keras.layers.Input(shape=(4,))

    # Hidden layers with dropout regularization
    x = tf.keras.layers.Dense(10, activation='relu')(inputs)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(10, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)

    # Outputs of the model
    weights = tf.keras.layers.Dense(3, activation='softmax', name='weights')(x)
    means = tf.keras.layers.Dense(3, name='means')(x)
    scales = tf.keras.layers.Dense(3, activation='softplus', name='scales')(x)

    # Create a distribution from the outputs
    dist = tfp.distributions.MixtureSameFamily(
        mixture_distribution=tfp.distributions.Categorical(probs=weights),
        components_distribution=tfp.distributions.Normal(loc=means, scale=scales))

    # Define the Keras model
    model = tf.keras.Model(inputs=inputs, outputs=dist)

    return model

# Create a TensorFlow Dataset object for the Iris data
data = tf.data.Dataset.from_tensor_slices((iris.values.astype(np.float32),))

# Create a TensorFlow Probability VI object for variational inference
vi = tfp.vi.fit_surrogate_posterior(
    target_log_prob_fn=lambda *args: model_fn().log_prob(args),
    surrogate_posterior=variational_fn(),
    optimizer=tf.optimizers.Adam(learning_rate=0.01),
    num_steps=1000,
    dataset=data)

# Extract the posterior distribution over the parameters of the model
weights, means, scales = vi.surrogate_posterior.sample(1000).numpy().mean(axis=0)

# Print the results
print('Inferred weights:', weights)
print('Inferred means:', means)
print('Inferred scales:', scales)


ValueError: Output tensors of a Functional model must be the output of a TensorFlow `Layer` (thus holding past layer metadata). Found: tfp.distributions.MixtureSameFamily("MixtureSameFamily", batch_shape=[?], event_shape=[], dtype=float32)